In [1]:
# Install needed stuff to the Jupyter and kernel environment
# Need to install only once, so commented out for cleaner output
# %pip install pyarrow
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install pyfinance
# %pip install yfinance
# %pip install yahoo-fin

In [2]:
# Import needed stuff
import sys
import pyarrow.feather as feather
import math
import pandas
import numpy as np
import matplotlib.pyplot as plt
from pyfinance.options import BSM
import yfinance as yf
from datetime import datetime, timedelta, date
from yahoo_fin import options as op

# Hedging assigment

# Fetch SP500 index and option data via ```yfinance```

### Data fetching and wrangling

In [3]:
# Get data from Yahoo Finance
def get_historical_data_and_option_expirations(ticker):
    ticker_data = yf.Ticker(ticker)
    
    # Get expiration dates of the options to a dataframe
    expiration_dates = pandas.DataFrame(data=ticker_data.options)
    
    return ticker_data, expiration_dates

In [4]:
def get_option_chain_by_expiration(data, expiration_date):
    # Get the option chain for the data
    option_chain_for_expiration = data.option_chain(expiration_date)
    
    # We will use call options for the hedging assignment, construct them to a dataframe
    opt_calls = pandas.DataFrame(option_chain_for_expiration.calls)
    
    # Get also the Put options and construct them to a dataframe
    # Optional and not needed yet in the assignment
    opt_puts = pandas.DataFrame(option_chain_for_expiration.puts)

    return opt_calls, opt_puts 
    

In [5]:
def write_dataframes_to_csv(df, file_name):
    df.to_csv(str(file_name) + '.csv', sep='\t', index=False)

In [6]:
# SPY stands for Standard & Poors

# Dataframe of historical data, list of expiration dates
ticker, option_expirations = get_historical_data_and_option_expirations('SPY')

# Take the first option, which expires
option_expiration_date = option_expirations.iloc[0][0]

# Dataframe of calls and dataframe of puts for a single expiration date
calls, puts = get_option_chain_by_expiration(ticker, option_expiration_date)

write_dataframes_to_csv(pandas.DataFrame(ticker.history(period='max'), 'historical_data_all'))
write_dataframes_to_csv(calls, 'calls_1')
write_dataframes_to_csv(puts, 'puts_1')


TypeError: Index(...) must be called with a collection of some kind, 'historical_data_all' was passed

## The hedging function itself

In [42]:
def delta_hedge():
    expirations = pandas.read_csv('sp500_expiration_dates_2023-11-10 11:47:07.709639.csv', sep='\t')
    calls = pandas.read_csv('sp500_calls_2023-11-10 11:47:07.714281.csv', sep='\t')

    # This option has the last price and strike price very close to each other
    # You can browse the option data to confirm
    option = calls.iloc[8] 
    print(option)

    # Get the days to maturity
    days_to_maturity = datetime(2023, 12, 29) - datetime.today()
    
    # Get time to maturity in years
    t = int(days_to_maturity.days)/365
    
    # Debug print
    print(t)
    print("Days to maturity in days is", days_to_maturity.days) # 49 days_to_maturity
    
    # time at t = 0, the date when the date was pulled
    t0 = datetime(2023, 11, 9).date()
    print("Time at t = 0 aka when data was fetched", t0)
    
    # call option with price C0 at time t0
    c0 = option['lastPrice']
    print('Price of Call option c0 in time t0', c0)

    # Strike price
    strike = option['strike']
    print('Strike price is: ', strike)
    
    # US treasury 10-year bond yield?
    r = 0.05
    
    # What would be a good value? 0.2 is used in the exercies
    sigma = 0.2
    
    # Value of the BSM for the call option c0 at time t0
    val = BSM(kind='call', S0=c0, K=strike, T=t, r=r, sigma=sigma).value() 
    print('value from BSM', val)
    
    # Delta of the BSM for the call option c0 at time t0
    delta = BSM(kind='call', S0=c0, K=strike, T=t, r=r, sigma=sigma).delta()
    print('Delta from BSM', delta)
    
    # Construct two portfolios:
    # OP with a long position in a call option
    # RE which is a short position of number of delta of the underlying asset
    
    # OP
    op = c0
    print('Long position in a call option c0 is ', c0)
    
    # RE
    re = -1*delta * c0
    print('Short position re ', re)

    # Portfolio value according to lecture 7
    portfolio_value = c0 - delta*c0
    print('Portfolio value ', portfolio_value)

    option_quantity = 1 #One call option ?

    pandas.to_datetime(calls['lastTradeDate'])
    sorted_calls = calls.sort_values(by=['lastTradeDate'])
    # sorted_calls.to_csv('sp500_calls_sorted_by_date_' + str(datetime.today()) + '.csv', sep='\t', index=False)

    # Changes in the portfolio OP are neutralized by oppsite changes in the replicatin portfolio re
    # rehedge every second day
    # Compute how much every portfolio has changed
    # Count mean squared error

In [ ]:
delta_hedge()